1. Split into train and test data
2. Train model on train data normally
3. Take test data and duplicate into test prime 
4. Drop first visit from test prime data
5. Get predicted delta from test prime data. Compare to delta from test data. We know the difference (epsilon) because we dropped actual visits. What percent of time is test delta < test prime delta? 
6. Restrict it only to patients with lot of visits. Is this better?

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pickle

def clean_plot():
    ax = plt.subplot(111)    
    ax.spines["top"].set_visible(False)    
    ax.spines["bottom"].set_visible(False)    
    ax.spines["right"].set_visible(False)    
    ax.spines["left"].set_visible(False)    
    
    ax.get_xaxis().tick_bottom()    
    ax.get_yaxis().tick_left()   
    plt.grid()

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
#           'figure.figsize': (10,6),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

In [ ]:
import sys
import torch

sys.path.append('../data')
from load import chf
from data_utils import parse_data
from synthetic_data import load_piecewise_synthetic_data


sys.path.append('../model')
from models import Sublign
from run_experiments import get_hyperparameters


In [ ]:
def make_test_prime(test_data_dict_raw, drop_first_T=1.):
    # drop first year
    test_data_dict = copy.deepcopy(test_data_dict_raw)
    eps_lst        = list()
    
    X = test_data_dict['obs_t_collect']
    Y = test_data_dict['Y_collect']
    M = test_data_dict['mask_collect']
    
    N_patients = X.shape[0]
    N_visits   = X.shape[1]
    
    for i in range(N_patients):
        eps_i = X[i,1,0] - X[i,0,0]
        
        first_visit = X[i,1,0]
        # move all visits down (essentially destroying the first visit)
        for j in range(N_visits-gap):
            
            X[i,j,0] = X[i,j+gap,0] - first_visit
            Y[i,j,:] = Y[i,j+gap,:]
            M[i,j,:] = M[i,j+gap,:]
        
        for g in range(1,gap+1):
            X[i,N_visits-g,0] = int(-1000)
            Y[i,N_visits-g,:] = int(-1000)
            M[i,N_visits-g,:] = 0.
        
        eps_lst.append(eps_i)
    return test_data_dict, eps_lst

In [ ]:
data       = chf()
max_visits = 38
shuffle    = True
num_output_dims = data.shape[1] - 4

data_loader, collect_dict, unique_pid = parse_data(data.values, max_visits=max_visits)
train_data_loader, train_data_dict, test_data_loader, test_data_dict, test_pid, unique_pid = parse_data(data.values, 
                                                                                                        max_visits=max_visits, test_per=0.2, 
                                                                                                        shuffle=shuffle)

# model = Sublign(10, 20, 50, dim_biomarkers=num_output_dims, sigmoid=True, reg_type='l1', auto_delta=True, 
#                 max_delta=5, learn_time=True, device=torch.device('cuda'))
# #         model.fit(data_loader, data_loader, args.epochs, 0.01, verbose=args.verbose,fname='runs/chf.pt',eval_freq=25)

# fname='../model/chf_good.pt'
# model.load_state_dict(torch.load(fname,map_location=torch.device('cuda')))


test_p_data_dict, eps_lst = make_test_prime(test_data_dict, gap=1)


# test_deltas   = model.get_deltas(test_data_dict).detach().numpy()
# test_p_deltas = model.get_deltas(test_p_data_dict).detach().numpy()

In [ ]:
print(num_output_dims)

In [ ]:
# def make_test_prime(test_data_dict_raw, drop_first_T=1.):
drop_first_T = 0.5
# drop first year

test_data_dict_new = copy.deepcopy(test_data_dict)
eps_lst        = list()

X = test_data_dict_new['obs_t_collect']
Y = test_data_dict_new['Y_collect']
M = test_data_dict_new['mask_collect']

N_patients = X.shape[0]
N_visits   = X.shape[1]

remove_idx = list()

X[X == -1000] = np.nan

for i in range(N_patients):
    N_visits_under_thresh = (X[i] < 0.5).sum()
    gap = N_visits_under_thresh
    
    first_valid_visit     = X[i,N_visits_under_thresh,0]
    
    eps_i = X[i,N_visits_under_thresh,0]
    
    for j in range(N_visits-N_visits_under_thresh):
        X[i,j,0] = X[i,j+gap,0] - first_valid_visit
        Y[i,j,:] = Y[i,j+gap,:]
        M[i,j,:] = M[i,j+gap,:]

    for g in range(1,N_visits_under_thresh+1):
        X[i,N_visits-g,0] = np.nan
        Y[i,N_visits-g,:] = np.nan
        M[i,N_visits-g,:] = 0.

    if np.isnan(X[i]).all():
        remove_idx.append(i)
    else:
        eps_lst.append(eps_i)

keep_idx = [i for i in range(N_patients) if i not in remove_idx]
X = X[keep_idx]
Y = Y[keep_idx]
M = M[keep_idx]

print('Removed %d entries' % len(remove_idx))
X[np.isnan(X)] = -1000

#     eps_lst.append(eps_i)
# return test_data_dict_new, eps_lst

In [ ]:
eps_lst

In [ ]:
X[0]

In [ ]:
first_valid_visit

In [ ]:
test_data_dict_new = copy.deepcopy(test_data_dict)

X = test_data_dict_new['obs_t_collect']
Y = test_data_dict_new['Y_collect']
M = test_data_dict_new['mask_collect']
X[X == -1000] = np.nan

i = 1
N_visits_under_thresh = (X[i] < 0.5).sum()



# for j in range(N_visits-N_visits_under_thresh):
#     X[i,j,0] = X[i,j+gap,0] - first_visit
#     Y[i,j,:] = Y[i,j+gap,:]
#     M[i,j,:] = M[i,j+gap,:]

# for g in range(1,N_visits_under_thresh+1):
#     X[i,N_visits-g,0] = np.nan
#     Y[i,N_visits-g,:] = np.nan
#     M[i,N_visits-g,:] = 0.

# if np.isnan(X[i]).all():
#     print('yes')
#     remove_idx.append(i)


In [ ]:
(X[1] < 0.5).sum()

In [ ]:
N_visits_under_thresh

In [ ]:
N_visits_under_thresh

In [ ]:
len(remove_idx)

In [ ]:
X[X == -1000] = np.nan
for i in range(10):
    print(X[i].flatten())

In [ ]:
remove_idx

In [ ]:
X[0][:10]

In [ ]:
plt.hist(X.flatten())

In [ ]:
X.max()

In [ ]:
Y[1][:10]

In [ ]:
test_data_dict_new['']

In [ ]:
f = open('chf_experiment_results.pk', 'rb')
results = pickle.load(f)
test_deltas    = results['test_deltas']
test_p_deltas  = results['test_p_deltas']
eps_lst        = results['eps_lst']
test_data_dict = results['test_data_dict']
f.close()

In [ ]:
test_data_dict['obs_t_collect'][0].shape

In [ ]:
# get num of visits per patient
num_visits_patient_lst = list()
for i in test_data_dict['obs_t_collect']:
    num_visits = (i!=-1000).sum()
    num_visits_patient_lst.append(num_visits)

num_visits_patient_lst = np.array(num_visits_patient_lst)

In [ ]:
freq_visit_idx = np.where(num_visits_patient_lst > 10)[0]

In [ ]:
test_p_deltas[freq_visit_idx]

In [ ]:
test_deltas[freq_visit_idx]

In [ ]:
np.mean(np.array(test_p_deltas - test_deltas) > 0)

In [ ]:
test_p_deltas[:20]

In [ ]:
clean_plot()
plt.plot(eps_lst, test_p_deltas - test_deltas, '.')
plt.xlabel('Actual eps')
plt.ylabel('Estimated eps')
#     plt.savefig('')

In [ ]:
import copy 

def make_test_prime(test_data_dict_raw, gap=1):
    test_data_dict = copy.deepcopy(test_data_dict_raw)
    eps_lst        = list()
    
    X = test_data_dict['obs_t_collect']
    Y = test_data_dict['Y_collect']
    M = test_data_dict['mask_collect']
    
    N_patients = X.shape[0]
    N_visits   = X.shape[1]
    
    for i in range(N_patients):
        eps_i = X[i,1,0] - X[i,0,0]
        
        first_visit = X[i,1,0]
        # move all visits down (essentially destroying the first visit)
        for j in range(N_visits-gap):
            
            X[i,j,0] = X[i,j+gap,0] - first_visit
            Y[i,j,:] = Y[i,j+gap,:]
            M[i,j,:] = M[i,j+gap,:]
        
        for g in range(1,gap+1):
            X[i,N_visits-g,0] = int(-1000)
            Y[i,N_visits-g,:] = int(-1000)
            M[i,N_visits-g,:] = 0.
        
        eps_lst.append(eps_i)
    return test_data_dict, eps_lst

In [ ]:
t_prime_dict, eps_lst = make_test_prime(test_data_dict)

In [ ]:
t_prime_dict['Y_collect'][1,:,0]

In [ ]:
test_data_dict['Y_collect'][1,:,0]

## Plot successful model

In [ ]:
import argparse
import numpy as np
import pickle
import sys
import torch
import copy

from scipy.stats import pearsonr
import matplotlib.pyplot as plt

from run_experiments import get_hyperparameters
from models import Sublign

sys.path.append('../data')

from data_utils import parse_data
from load import load_data_format

sys.path.append('../evaluation')
from eval_utils import swap_metrics

In [ ]:
train_data_dict['Y_collect'].shape


In [ ]:
train_data_dict['t_collect'].shape

In [ ]:
new_Y = np.zeros((600,101,3))

In [ ]:
val_idx_dict = {'%.1f' % j: i for i,j in enumerate(np.linspace(0,10,101))}

In [ ]:
train_data_dict['obs_t_collect'].max()

In [ ]:
rounded_t = np.round(train_data_dict['t_collect'],1)
N, M, _ = rounded_t.shape

for i in range(N):
    for j in range(M):
        val = rounded_t[i,j,0]
#         try:
        idx = val_idx_dict['%.1f' % val]
        for k in range(3):
            new_Y[i,idx,k] = train_data_dict['Y_collect'][i,j,k]
#         except:
#             print(val)

In [ ]:
new_Y.shape

In [ ]:
(new_Y == 0).sum() / (600*101*3)

In [ ]:
# save the files for comparing against SPARTan baseline

for i in range(3):
    a = new_Y[:,:,i]
    np.savetxt("data1_dim%d.csv" % i, a, deliREDACTEDer=",")

In [ ]:
true_labels = train_data_dict['s_collect'][:,0]
guess_labels = np.ones(600)

adjusted_rand_score(true_labels,guess_labels)

In [ ]:
from sklearn.metrics import adjusted_rand_score
# a.shape

In [ ]:
data_format_num = 1
# C, d_s, d_h, d_rnn, reg_type, lr = get_hyperparameters(data_format_num)
anneal, b_vae, C, d_s, d_h, d_rnn, reg_type, lr = get_hyperparameters(data_format_num)
C
data = load_data_format(data_format_num, 0, cache=True)

train_data_loader, train_data_dict, _, _, test_data_loader, test_data_dict, valid_pid, test_pid, unique_pid = parse_data(data.values, max_visits=4, test_per=0.2, valid_per=0.2, shuffle=False)

model  = Sublign(d_s, d_h, d_rnn, dim_biomarkers=3, sigmoid=True, reg_type='l1', auto_delta=False, max_delta=0, learn_time=False, beta=0.00)
model.fit(train_data_loader, test_data_loader, 800, lr, fname='runs/data%d_chf_experiment.pt' % (data_format_num), eval_freq=25)

z = model.get_mu(train_data_dict['obs_t_collect'], train_data_dict['Y_collect'])
# fname='runs/data%d_chf_experiment.pt' % (data_format_num)
# model.load_state_dict(torch.load(fname))
nolign_results = model.score(train_data_dict, test_data_dict)
print('ARI: %.3f' % nolign_results['ari'])

In [ ]:
print(anneal, b_vae, C, d_s, d_h, d_rnn, reg_type, lr)

In [ ]:
data_format_num = 1
# C, d_s, d_h, d_rnn, reg_type, lr = get_hyperparameters(data_format_num)
anneal, b_vae, C, d_s, d_h, d_rnn, reg_type, lr = get_hyperparameters(data_format_num)

model  = Sublign(d_s, d_h, d_rnn, dim_biomarkers=3, sigmoid=True, reg_type='l1', auto_delta=True, max_delta=5, learn_time=True, beta=0.01)
model.fit(train_data_loader, test_data_loader, 800, lr, fname='runs/data%d.pt' % (data_format_num), eval_freq=25)

z = model.get_mu(train_data_dict['obs_t_collect'], train_data_dict['Y_collect'])
# fname='runs/data%d_chf_experiment.pt' % (data_format_num)
# model.load_state_dict(torch.load(fname))
results = model.score(train_data_dict, test_data_dict)
print('ARI: %.3f' % results['ari'])

In [ ]:
# model  = Sublign(d_s, d_h, d_rnn, dim_biomarkers=3, sigmoid=True, reg_type='l1', auto_delta=True, max_delta=5, learn_time=True, b_vae=0.)
# model.fit(train_data_loader, test_data_loader, 800, lr, fname='runs/data%d_chf_experiment.pt' % (data_format_num), eval_freq=25)

# z = model.get_mu(train_data_dict['obs_t_collect'], train_data_dict['Y_collect'])
# # fname='runs/data%d_chf_experiment.pt' % (data_format_num)
# # model.load_state_dict(torch.load(fname))
# results = model.score(train_data_dict, test_data_dict)
# print('ARI: %.3f' % results['ari'])


In [ ]:
# Visualize latent space (change configs above)
X = test_data_dict['obs_t_collect']
Y = test_data_dict['Y_collect']
M = test_data_dict['mask_collect']


test_z, _ = model.get_mu(X,Y)
test_z    = test_z.detach().numpy()

test_subtypes = test_data_dict['s_collect']

from sklearn.manifold import TSNE
z_tSNE = TSNE(n_components=2).fit_transform(test_z)

test_s0_idx = np.where(test_subtypes==0)[0]
test_s1_idx = np.where(test_subtypes==1)[0]

clean_plot()
plt.plot(z_tSNE[test_s0_idx,0],z_tSNE[test_s0_idx,1],'.')
plt.plot(z_tSNE[test_s1_idx,0],z_tSNE[test_s1_idx,1],'.')

# plt.title('\nNELBO (down): %.3f, ARI (up): %.3f\n Config: %s\nColors = true subtypes' % 
#           (nelbo, ari, configs))
plt.show()

In [ ]:
def sigmoid_f(x, beta0, beta1):
    result = 1. / (1+np.exp(-(beta0 + beta1*x)))
    return result

true_betas = [[[-4, 1],
        [-1,1.],
        [-8,8]
        ],
        [
        [-1,1.],
        [-8,8],
        [-25, 3.5]
        ]]

In [ ]:
# xs = np.linspace(0,10,100)

for dim_i in range(3):
    xs = np.linspace(0,10,100)
    
    plt.figure()
    clean_plot()
    plt.grid(True)
    ys = [sigmoid_f(xs_i, true_betas[0][dim_i][0], true_betas[0][dim_i][1]) for xs_i in xs]
    plt.plot(xs,ys, ':', color='gray', linewidth=5, label='True function')

    ys = [sigmoid_f(xs_i, true_betas[1][dim_i][0], true_betas[1][dim_i][1]) for xs_i in xs]
    plt.plot(xs,ys, ':', color='gray', linewidth=5)

    for subtype_j in range(2):
        

        xs = np.linspace(0,10,100)
        ys = [sigmoid_f(xs_i, nolign_results['cent_lst'][subtype_j,dim_i,0], 
                                nolign_results['cent_lst'][subtype_j,dim_i,1]) for xs_i in xs]
        if subtype_j == 0:
            plt.plot(xs,ys,linewidth=4, label='SubNoLign subtype', linestyle='-.', color='tab:green')
        else:
            plt.plot(xs,ys,linewidth=4, linestyle='--', color='tab:green')

        ys = [sigmoid_f(xs_i, results['cent_lst'][subtype_j,dim_i,0], 
                                results['cent_lst'][subtype_j,dim_i,1]) for xs_i in xs]
        if subtype_j == 0:
            plt.plot(xs,ys,linewidth=4, label='SubLign subtype', linestyle='-', color='tab:purple')
        else:
            plt.plot(xs,ys,linewidth=4, linestyle='-', color='tab:purple')


            
    plt.xlabel('Disease stage')
    plt.ylabel('Biomarker')
    plt.legend()
    plt.savefig('subnolign_data1_subtypes_dim%d.pdf' % dim_i, bbox_inches='tight')
    

In [ ]:
# # number dimensions
# fig, axs = plt.subplots(1,3, figsize=(8,4))
# for dim_i in range(3):
#     ax = axs[dim_i]
#     # number subtypes
#     for subtype_j in range(2):
#         xs = np.linspace(0,10,100)
#         ys = [sigmoid_f(xs_i, model1_results['cent_lst'][subtype_j,dim_i,0], 
#                         model1_results['cent_lst'][subtype_j,dim_i,1]) for xs_i in xs]
                

#         ax.plot(xs,ys)
#         ys = [sigmoid_f(xs_i, true_betas[0][dim_i][0], true_betas[0][dim_i][1]) for xs_i in xs]
#         ax.plot(xs,ys, color='gray')
        
#         ys = [sigmoid_f(xs_i, true_betas[1][dim_i][0], true_betas[1][dim_i][1]) for xs_i in xs]
#         ax.plot(xs,ys, color='gray')
        
# fig.suptitle('True data generating function (gray), learned models (orange, blue)')
# plt.savefig('learned_models.pdf',bbox_inches='tight')

## Plot CHF Delta distributions

In [ ]:
data = pickle.load(open('../clinical_runs/chf_v3_1000.pk', 'rb'))
clean_plot()
plt.hist(data['deltas'], bins=20)
plt.xlabel('Inferred Alignment $\delta_i$ Value')
plt.ylabel('Number Heart Failure Patients')
plt.savefig('Delta_dist_chf.pdf', bbox_inches='tight')

## Make piecewise data to measure model misspecification

In [ ]:
from scipy import interpolate

In [ ]:
x = np.arange(0, 2*np.pi+np.pi/4, 2*np.pi/8)
y = np.sin(x)
tck = interpolate.splrep(x, y, s=0)
xnew = np.arange(0, 2*np.pi, np.pi/50)
ynew = interpolate.splev(xnew, tck, der=0)

In [ ]:
xvals = np.array([9.3578453 , 4.9814664 , 7.86530539, 8.91318433, 2.00779188])[sort_idx]
yvals = np.array([0.35722491, 0.12512101, 0.20054626, 0.38183604, 0.58836923])[sort_idx]
tck = interpolate.splrep(xvals, yvals, s=0)

In [ ]:
y

In [ ]:
N_subtypes,D,N_pts,_ = subtype_points.shape

fig, axes = plt.subplots(ncols=3,nrows=1)

for d, ax in enumerate(axes.flat):
#     ax.set_xlim(0,10)
#     ax.set_ylim(0,1)
    for k in range(N_subtypes):
        xs = subtype_points[k,d,:,0]
        ys = subtype_points[k,d,:,1]
        sort_idx = np.argsort(xs)
        ax.plot(xs[sort_idx],ys[sort_idx])

plt.show()

# for d in range(D):
    

In [ ]:
%%time
N_epochs    = 800
N_trials    = 5
use_sigmoid = True

sublign_results = {
    'ari':[],
    'pear': [],
    'swaps': []
}
subnolign_results = {'ari': []}

for trial in range(N_trials):
    data_format_num = 1
    # C, d_s, d_h, d_rnn, reg_type, lr = get_hyperparameters(data_format_num)
    anneal, b_vae, C, d_s, d_h, d_rnn, reg_type, lr = get_hyperparameters(data_format_num)
    # C
    # data = load_data_format(data_format_num, 0, cache=True)

    use_sigmoid = False

    data, subtype_points = load_piecewise_synthetic_data(subtypes=2, increasing=use_sigmoid, 
                            D=3, N=2000,M=4, noise=0.25, N_pts=5)

    train_data_loader, train_data_dict, _, _, test_data_loader, test_data_dict, valid_pid, test_pid, unique_pid = parse_data(data.values, max_visits=4, test_per=0.2, valid_per=0.2, shuffle=False)

    model  = Sublign(d_s, d_h, d_rnn, dim_biomarkers=3, sigmoid=use_sigmoid, reg_type='l1', 
                     auto_delta=False, max_delta=5, learn_time=True, beta=1.)
    model.fit(train_data_loader, test_data_loader, N_epochs, lr, fname='runs/data%d_spline.pt' % (data_format_num), eval_freq=25)

    # z = model.get_mu(train_data_dict['obs_t_collect'], train_data_dict['Y_collect'])
    # fname='runs/data%d_chf_experiment.pt' % (data_format_num)
    # model.load_state_dict(torch.load(fname))
    results = model.score(train_data_dict, test_data_dict)
    print('Sublign results: ARI: %.3f; Pear: %.3f; Swaps: %.3f' % (results['ari'],results['pear'],results['swaps']))
    sublign_results['ari'].append(results['ari'])
    sublign_results['pear'].append(results['pear'])
    sublign_results['swaps'].append(results['swaps'])
    
    model  = Sublign(d_s, d_h, d_rnn, dim_biomarkers=3, sigmoid=use_sigmoid, reg_type='l1', 
                     auto_delta=False, max_delta=0, learn_time=False, beta=1.)
    model.fit(train_data_loader, test_data_loader, N_epochs, lr, fname='runs/data%d_spline.pt' % (data_format_num), eval_freq=25)
    nolign_results = model.score(train_data_dict, test_data_dict)
    print('SubNoLign results: ARI: %.3f' % (nolign_results['ari']))
    subnolign_results['ari'].append(nolign_results['ari'])

In [ ]:
data_str = 'Increasing' if use_sigmoid else 'Any'
print('SubLign-%s & %.2f $\\pm$ %.2f & %.2f $\\pm$ %.2f & %.2f $\\pm$ %.2f \\\\' % (
    data_str,
    np.mean(sublign_results['ari']), np.std(sublign_results['ari']),
    np.mean(sublign_results['pear']), np.std(sublign_results['pear']),
    np.mean(sublign_results['swaps']), np.std(sublign_results['swaps'])
))

print('SubNoLign-%s & %.2f $\\pm$ %.2f & -- &  --  \\\\' % (
    data_str,
    np.mean(sublign_results['ari']), np.std(sublign_results['ari']),
))

In [ ]:
results = model.score(train_data_dict, test_data_dict)
print('Sublign results: ARI: %.3f; Pear: %.3f; Swaps: %.3f' % (results['ari'],results['pear'],results['swaps']))
